# V2

## Polarity issues fixed

In [1]:
import mne
import pandas as pd
import numpy as np
from pathlib import Path

# --- 1. Setup Paths ---
scripts_folder = Path.cwd()

# Go up two levels to find the root, then down to data
root_folder = scripts_folder.parent.parent
dataset_path = root_folder / 'data' / 'ds003838'

# Create output folder
output_folder = scripts_folder / 'rest_data_v2'
output_folder.mkdir(exist_ok=True)

print(f"📂 Reading data from: {dataset_path}")
print(f"💾 Saving output to:  {output_folder}")
print()

# Exclude subjects with missing physiological data
excluded_subjects = {
    'sub-013', 'sub-014', 'sub-015', 'sub-016', 'sub-017', 'sub-018',
    'sub-019', 'sub-020', 'sub-021', 'sub-022', 'sub-023', 'sub-024',
    'sub-025', 'sub-026', 'sub-027', 'sub-028', 'sub-029', 'sub-030',
    'sub-031', 'sub-037', 'sub-066', 'sub-094'
}

task = 'rest'

# --- 2. Find All Subjects ---
all_subject_folders = sorted([f for f in dataset_path.glob('sub-*') if f.is_dir()])
subject_folders = [f for f in all_subject_folders if f.name not in excluded_subjects]
print(f"🔍 Found {len(all_subject_folders)} subjects total")
print(f"❌ Excluding {len(excluded_subjects)} subjects with missing data")
print(f"✅ Processing {len(subject_folders)} subjects with complete data")
print()

# --- 3. Process Each Subject ---
successful = 0
failed = 0
skipped = 0

for subject_path in subject_folders:
    subject_id = subject_path.name
    ecg_folder = subject_path / 'ecg'

    # Check if ECG folder exists
    if not ecg_folder.exists():
        print(f"⏭️  {subject_id}: No ECG folder, skipping")
        skipped += 1
        continue

    # Build file paths
    set_filename = f"{subject_id}_task-{task}_ecg.set"
    set_path = ecg_folder / set_filename

    if not set_path.exists():
        print(f"⏭️  {subject_id}: No .set file, skipping")
        skipped += 1
        continue

    print(f"⏳ {subject_id}: Loading...")
    try:
        # Load the binary file with MNE
        raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)

        # Sampling frequency (should be 1000 Hz for this dataset)
        sfreq = raw.info["sfreq"]

        # Ensure ECG channel is typed correctly and pick it
        ecg_ch = "ECG"
        raw.set_channel_types({ecg_ch: "ecg"})
        ecg_picks = mne.pick_channels(raw.info["ch_names"], include=[ecg_ch])
        if len(ecg_picks) == 0:
            raise RuntimeError("No ECG channel found in Raw object")

        # Get data in Volts (MNE's internal unit)
        ecg_data, times = raw.get_data(picks=ecg_picks, return_times=True)
        ecg = ecg_data[0]  # shape: (n_samples,)

        # --- Decide polarity using detrend + z-score ---
        ecg_detrended = mne.filter.detrend(ecg, order=1)
        ecg_z = (ecg_detrended - np.mean(ecg_detrended)) / np.std(ecg_detrended)
        pos_energy = np.mean(ecg_z[ecg_z > 0] ** 2) if np.any(ecg_z > 0) else 0
        neg_energy = np.mean(ecg_z[ecg_z < 0] ** 2) if np.any(ecg_z < 0) else 0
        flip = neg_energy > pos_energy

        # Polarity-corrected versions
        ecg_mod = ecg_z if not flip else -ecg_z          # detrended + z-scored
        ecg_nk = ecg if not flip else -ecg               # raw volts, flipped if needed

        # Build DataFrame
        df = pd.DataFrame({
            "time":    times,      # seconds
            "ecg_raw": ecg,     # original volts
            "ecg_mod": ecg_mod, # detrended, z-scored, polarity-fixed
            "ecg_nk":  ecg_nk    # raw volts, polarity-fixed
        })

        # Save as Parquet
        parquet_filename = f"{subject_id}_task-{task}_ecg_rest.parquet"
        output_path = output_folder / parquet_filename
        df.to_parquet(output_path, index=False)

        print(f"✅ {subject_id}: Success! ({df.shape[0]} samples)")
        print(f"   Sampling rate: {sfreq} Hz (use this in NeuroKit2)")
        successful += 1

    except Exception as e:
        print(f"❌ {subject_id}: Error - {e}")
        failed += 1

# --- 4. Summary ---
print()
print("=" * 60)
print("🎯 Processing Complete!")
print(f"   ✅ Successful: {successful}")
print(f"   ❌ Failed:     {failed}")
print(f"   ⏭️  Skipped:    {skipped}")
print(f"   📊 Total:      {len(subject_folders)}")
print("=" * 60)

📂 Reading data from: /home/martin/RESEARCH/thesis/code/brain_heart_psv_sdg/data/ds003838
💾 Saving output to:  /home/martin/RESEARCH/thesis/code/brain_heart_psv_sdg/code/digitspan/rest_data_v2

🔍 Found 86 subjects total
❌ Excluding 22 subjects with missing data
✅ Processing 64 subjects with complete data

⏳ sub-032: Loading...


/home/martin/miniforge3/envs/brain-heart/lib/python3.11/site-packages/pymatreader/utils.py:179: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn(
/tmp/ipykernel_9929/180422332.py:66: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
/tmp/ipykernel_9929/180422332.py:66: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)


✅ sub-032: Success! (227081 samples)
   Sampling rate: 1000.0 Hz (use this in NeuroKit2)
⏳ sub-033: Loading...


/tmp/ipykernel_9929/180422332.py:66: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
/tmp/ipykernel_9929/180422332.py:66: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)


✅ sub-033: Success! (185921 samples)
   Sampling rate: 1000.0 Hz (use this in NeuroKit2)
⏳ sub-034: Loading...


...

/tmp/ipykernel_9929/180422332.py:66: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
/tmp/ipykernel_9929/180422332.py:66: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)


✅ sub-096: Success! (278841 samples)
   Sampling rate: 1000.0 Hz (use this in NeuroKit2)
⏳ sub-097: Loading...


/tmp/ipykernel_9929/180422332.py:66: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
/tmp/ipykernel_9929/180422332.py:66: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)


✅ sub-097: Success! (251281 samples)
   Sampling rate: 1000.0 Hz (use this in NeuroKit2)
⏳ sub-098: Loading...
✅ sub-098: Success! (229921 samples)
   Sampling rate: 1000.0 Hz (use this in NeuroKit2)

🎯 Processing Complete!
   ✅ Successful: 64
   ❌ Failed:     0
   ⏭️  Skipped:    0
   📊 Total:      64


/tmp/ipykernel_9929/180422332.py:66: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
/tmp/ipykernel_9929/180422332.py:66: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_path, preload=True, verbose=False)
